In [24]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error

Подготовка данных

In [2]:
df = pd.read_csv('../../data/20scores.csv')
df.head()

,revenue,score0,score1,score2,score3,score4,score5,score6,score7,score8,...,score10,score11,score12,score13,score14,score15,score16,score17,score18,score19
0,30698.74,0.067763,1.762772,0.211119,0.619655,-1.586284,0.051320,-0.529940,-0.177908,-0.387431,...,-1.246132,-1.817742,-0.189583,-1.636507,0.088087,0.481573,2.243539,0.330061,-1.019798,-1.409513
1,46813.75,0.943657,-0.696100,3.503075,1.323145,-0.579567,-1.379598,0.013465,1.061996,-0.952645,...,1.115770,0.455824,0.109667,-0.790210,1.876311,0.610918,0.208647,1.505998,0.671251,0.315013
2,39493.35,-0.070838,-1.817580,-0.156724,-0.159741,-1.564338,-0.817489,0.125174,-1.053015,0.181246,...,-0.006912,-0.009129,-0.324959,1.040768,0.375134,-0.127496,1.435406,3.267201,-0.010580,-1.725084
3,48130.55,-0.133306,0.815997,2.261204,0.794839,-0.947440,0.049189,-0.042403,1.657086,-2.870217,...,2.183177,0.364106,0.686530,0.112862,1.840957,-1.003409,1.641141,1.469621,-1.839971,-0.456130
4,35129.09,0.452780,1.529394,0.085364,-0.787245,1.351148,-0.340774,0.354099,-0.319731,-0.776033,...,2.000792,1.035293,-0.719229,-3.073072,-0.078145,-0.154752,0.117198,0.226051,-1.447333,0.463009


In [3]:
x_cols = df.columns[1:]
X = df[x_cols]

y = df['revenue']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 10)

Модель

In [7]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [8]:
model.coef_

array([ 3.10465458e+01,  1.35929333e+00, -1.71996170e+01, -4.26396854e+00,
       -4.56514104e+00,  2.71178012e+01,  1.12523398e+01, -9.62768549e+00,
        1.28097189e+01, -3.82102937e+01, -3.92691076e+00, -4.49267755e+00,
        9.12581579e+03,  2.81237962e+01,  1.26722148e+01,  1.99096955e+01,
       -1.73401880e+01,  3.77047162e+03, -7.57356369e+00,  4.99844116e+03])

Расчет RFE

In [10]:
rfe = RFE(estimator=LinearRegression(), n_features_to_select=5)

In [11]:
rfe.fit(X_train, y_train)

RFE(estimator=LinearRegression(), n_features_to_select=5)

RFE выбрал только 5 важных функций

In [15]:
for featureNum in range(X_train.shape[1]):
  if rfe.support_[featureNum]:
    print(f"Feature: {X_train.columns[featureNum]}, Rank: {rfe.ranking_[featureNum]}")

Feature: score0, Rank: 1
Feature: score9, Rank: 1
Feature: score12, Rank: 1
Feature: score17, Rank: 1
Feature: score19, Rank: 1


Выбрать только те колонки, кот выбрал RFE

In [21]:
X_train_reduced = X_train[X_train.columns[rfe.support_]]
X_test_reduced = X_test[X_train.columns[rfe.support_]]

Новая модель

In [23]:
rfe_model = LinearRegression()
rfe_model.fit(X_train_reduced, y_train)

LinearRegression()

RSME для обычной модели

In [25]:
predictions = model.predict(X_test)
print(mean_squared_error(predictions, y_test)**0.5)

491.78833768572525


RSME для модели после RFE

In [26]:
rfe_predictions = rfe_model.predict(X_test_reduced)
print(mean_squared_error(rfe_predictions, y_test)**0.5)

487.618417198659


RFE упростил модель до 5 функций и получил более лкчшую оценку RSME